In [1]:
from transformers import BertTokenizerFast, TrainingArguments, Trainer, BertForTokenClassification, DataCollatorForTokenClassification
from datasets import load_dataset
from huggingface_hub import login
from dotenv import load_dotenv
import os

/home/david/grt/grt_reader/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
login(token=os.getenv("HUGGINGFACE_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/david/.cache/huggingface/token
Login successful


In [4]:
# Load the CoNLL-2003 dataset
dataset = load_dataset("conll2003")


In [5]:
# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

/home/david/grt/grt_reader/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
# Define label mapping
label_list = dataset["train"].features["ner_tags"].feature.names
label_to_id = {label: i for i, label in enumerate(label_list)}

# Tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply tokenization and label alignment
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


In [7]:
# Initialize the model
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_list))

# Define data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/david/grt/grt_reader/.venv/lib/python3.11/site-packages/accelerate/accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.159700,0.046178
2,0.025800,0.037219
3,0.015200,0.037509


TrainOutput(global_step=2634, training_loss=0.05129170377053541, metrics={'train_runtime': 4135.2545, 'train_samples_per_second': 10.186, 'train_steps_per_second': 0.637, 'total_flos': 1054683418795902.0, 'train_loss': 0.05129170377053541, 'epoch': 3.0})

In [8]:
repo_name = "darmendarizp/eafit-ner-v1"
trainer.save_model("./model")
trainer.model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors: 100%|██████████| 431M/431M [01:53<00:00, 3.78MB/s]   


CommitInfo(commit_url='https://huggingface.co/darmendarizp/eafit-ner-v1/commit/a256bc58245830f45b37a985014ed3a9f4c2749d', commit_message='Upload tokenizer', commit_description='', oid='a256bc58245830f45b37a985014ed3a9f4c2749d', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
results = trainer.evaluate()

In [11]:
print(results)

{'eval_loss': 0.03750857338309288, 'eval_runtime': 39.4151, 'eval_samples_per_second': 82.456, 'eval_steps_per_second': 5.176, 'epoch': 3.0}
